In [5]:
import ee
import geemap
import geopandas as gpd
import os
from shapely.ops import transform


# Earth Engine Initialization & Authentication
print("Initializing Google Earth Engine...")
try:
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine initialized successfully.")
except Exception as e:
    print("Earth Engine not initialized. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project='golf-mapper-463720')
    print("Earth Engine re-authenticated and initialized.")


try:
    account = ee.data.getAssetRoots()[0]['id']
    print(f"✅ Authenticated as: {account}")
except:
    print("❌ Still not authenticated")

Initializing Google Earth Engine...
Earth Engine initialized successfully.
❌ Still not authenticated




earthengine can only deal with (x,y) coordinates, and earth pro exports KML files with a Z coordinate. The code below removes the z coordinate. FIXME ADD TRY/CATCH BLOCK

In [2]:
# Read your KML
gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test3.kml')

# Function to remove Z coordinates
def remove_z_dimension(geom):
    if geom.has_z:
        return transform(lambda x, y, z=None: (x, y), geom)
    return geom

# Apply to all geometries
gdf['geometry'] = gdf.geometry.apply(remove_z_dimension)

# Verify it worked
print("After Z removal:")
print("Geometry types:", gdf.geometry.type.value_counts())
print("Sample geometry:", gdf.geometry.iloc[0])

# Now try the Earth Engine conversion
geometry = geemap.gdf_to_ee(gdf)
print("Success! Converted to Earth Engine geometry")




After Z removal:
Geometry types: Polygon    1
Name: count, dtype: int64
Sample geometry: POLYGON ((-95.37972929310104 37.65883757523466, -95.35579120123668 37.65832987171771, -95.35416643387549 37.66965512912511, -95.3815263901972 37.66997271533956, -95.37972929310104 37.65883757523466))
Success! Converted to Earth Engine geometry


In [3]:
import ee

# Force re-authentication
ee.Authenticate()
ee.Initialize()

# Then verify it worked
try:
    account = ee.data.getAssetRoots()[0]['id']
    print(f"✅ Authenticated as: {account}")
except:
    print("❌ Still not authenticated")

    
# start_date = '2023-01-01'
# end_date = '2023-12-31'

# # Get Sentinel-2 imagery
# sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
#     .filterDate(start_date, end_date) \
#     .filterBounds(geometry) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
#     .median()  # Get median composite

# # Clip the image to your polygon
# clipped_image = sentinel.clip(geometry)




# # More robust export
# try:
#     # Check if image has any data
#     pixel_count = clipped_image.select('B4').reduceRegion(
#         reducer=ee.Reducer.count(),
#         geometry=geometry,
#         scale=100,  # Coarser scale for counting
#         maxPixels=1e6
#     ).getInfo()
#     print("Pixel count:", pixel_count)
    
#     if pixel_count['B4'] > 0:
#         task = ee.batch.Export.image.toDrive(
#             image=clipped_image,
#             description='sentinel2_2023_clipped_v2',
#             folder='GEE_exports',
#             fileNamePrefix='sentinel2_2023_median_v2',
#             region=geometry,
#             scale=10,
#             crs='EPSG:4326',
#             maxPixels=1e9,
#             fileFormat='GeoTIFF'
#         )
#         task.start()
#         print(f"Task started: {task.id}")
#     else:
#         print("No valid pixels found in the region")
        
# except Exception as e:
#     print(f"Error: {e}")


# # Check your Google account info
# print("GEE Account:", ee.data.getAssetRoots()[0]['id'] if ee.data.getAssetRoots() else "No account found")

# # Try a very simple export with explicit parameters
# simple_task = ee.batch.Export.image.toDrive(
#     image=clipped_image.select('B4'),  # Just one band
#     description='simple_test',
#     fileNamePrefix='simple_test',
#     region=geometry,
#     scale=30,
#     maxPixels=1e6
# )
# simple_task.start()
# print(f"Simple task started: {simple_task.id}")
# # print(ee.data.getAssetRoots())


# # Try getting more detailed task info
# tasks = ee.batch.Task.list()
# for task in tasks[:5]:
#     if task.state == 'READY':
#         print(f"✅ Task ID: {task.id}")
#         print(f"Description: {task.config.get('description', 'N/A')}")
#         print(f"State: {task.state}")
#         print(f"Task type: {task.task_type}")
#         print(f"All available attributes: {dir(task)}")
#         print("=" * 50)




❌ Still not authenticated


In [ ]:
start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)


# More robust export
try:
    # Check if image has any data
    pixel_count = clipped_image.select('B4').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=100,  # Coarser scale for counting
        maxPixels=1e6
    ).getInfo()
    print("Pixel count:", pixel_count)
    
    if pixel_count['B4'] > 0:
        task = ee.batch.Export.image.toDrive(
            image=clipped_image,
            description='sentinel2_2023_clipped_v2',
            folder='GEE_exports',
            fileNamePrefix='sentinel2_2023_median_v2',
            region=geometry,
            scale=10,
            crs='EPSG:4326',
            maxPixels=1e9,
            fileFormat='GeoTIFF'
        )
        task.start()
        print(f"Task started: {task.id}")
    else:
        print("No valid pixels found in the region")
        
except Exception as e:
    print(f"Error: {e}")













# Download directly to local machine
# geemap.ee_export_image_to_drive(
#     clipped_image,
#     description='my_clipped_image',
#     folder='/Users/carrollcj/proj/golfmapper/exported_images',
#     region=geometry,
#     scale=10,
#     crs='EPSG:4326'
# )

# Export to Google Drive
task = ee.batch.Export.image.toDrive(
    image=clipped_image,
    description='sentinel2_2023_clipped',
    folder='GEE_exports',  # folder in your Drive
    fileNamePrefix='sentinel2_2023_median',
    region=geometry,
    scale=10,  # Sentinel-2 resolution
    crs='EPSG:4326',
    maxPixels=1e9,
    fileFormat='GeoTIFF'
)
task.start()

# Check if your task is still running
# tasks = ee.batch.Task.list()
# for task in tasks[:5]:  # Show last 5 tasks
#     print(f"Task ID: {task.id}")
#     print(f"Description: {task.config['description']}")
#     print(f"Status: {task.state}")
#     # print(f"Creation time: {task.creation_timestamp_ms}")
#     print("---")

# Get detailed export info
tasks = ee.batch.Task.list()
for task in tasks[:5]:
    if task.state == 'READY':
        print(f"✅ Task ID: {task.id}")
        print(f"Description: {task.config['description']}")
        print(f"Full config: {task.config}")
        print("=" * 50)

print(ee.data.getAssetRoots())


#print('Export task started. Check your Google Drive.')



#FUNCTIONAL CODE TO VISUALIZE A GEOMETRY ON THE MAP

In [ ]:
#FIXME for additional polygons its best to use geopandas.
#Need to integrate, but for now keep the interactive map

# Create an Interactive Map
Map = geemap.Map()
print("Interactive map object created.")

print("geemap version:", geemap.__version__)

display(Map)

# adds the kml file onto the map
Map.add_kml('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

In [ ]:
geometry = geemap.kml_to_ee('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')


start_date = '2023-01-01'
end_date = '2023-12-31'

# Get Sentinel-2 imagery
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(geometry) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()  # Get median composite

# Clip the image to your polygon
clipped_image = sentinel.clip(geometry)

# # Clip Sentinel imagery
# clipped_image = sentinel.clip(geometry)

In [ ]:
#checking to see the data shape


# gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

# gemoetry = geemap.gdf_to_ee(gdf)


import geopandas as gpd

gdf = gpd.read_file('/Users/carrollcj/proj/golfmapper/kml_files/test2.kml')

# Check the data structure
print("Shape:", gdf.shape)
print("Columns:", gdf.columns.tolist())
print("Geometry types:", gdf.geometry.type.value_counts())
print("CRS:", gdf.crs)
print("Sample data:")
print(gdf.head())

# Check for geometry issues
print("Null geometries:", gdf.geometry.isnull().sum())
print("Invalid geometries:", (~gdf.geometry.is_valid).sum())